In [1]:
# Import necessary libraries
import sys
from pathlib import Path
repo_root = Path("/home/ubuntu/michael/MSc-Machine-Learning-Project")
src_path = repo_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
import os
import numpy as np
import pandas as pd
import pyarrow.feather as feather
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [2]:
# Function to clean the equities datasets and select features
def clean_and_select_features_equities(df: pd.DataFrame, redundant_features: list) -> pd.DataFrame:
    """
    Takes a raw equities dataframe (with date on YYYY-MM-DD format) and returns a cleaned feature set after handling look-ahead bias and multicollinearity.
    Args:
        df: The raw input dataframe
        redundant_features:  list of redundant column names to drop
    Returns:
        A dataframe containing only the cleaned and selected features.
    """

    # Sort by date
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date').reset_index(drop=True)

    # Identify and shift "-F" columns to avoid look-ahead bias
    future_cols = [c for c in df.columns if '-F' in c]
    intl_indices = ['FTSE', 'GDAXI', 'FCHI', 'HSI', 'SSEC']
    col_to_shift = future_cols + [c for c in intl_indices if c in df.columns]
    print(f"\nShifting {len(col_to_shift)} columns by +1 day")

    for col in col_to_shift:
        df[col] = df[col].shift(1)

    # Define leaky feature columns to be dropped 
    leaky_momentum = ['mom', 'mom1', 'mom2', 'mom3']

     # Drop name of index (constant across all observations)
    name= ['Name']

    cols_to_drop = leaky_momentum + name + redundant_features
    # Combine all lists of columns to drop 
    cleaned_df = df.drop(columns=cols_to_drop, errors='ignore')

    print(f"Number of columns dropped: {len(set(cols_to_drop) & set(df.columns))}")

    return cleaned_df

In [3]:
# Redundant features
redundant_nyse = [
    'TE6', 'DTB6', 'DE4', 'TE5', 'DTB4WK', 'DAAA', 
    'DGS10', 'DE5', 'DTB3', 'DE6', 'EMA_20', 'CTB6M', 
    'CTB3M', 'EMA_50', 'CTB1Y', 'TE2', 'GSPC', 'DGS5', 
    'S&P-F', 'FCHI', 'EMA_200', 'GDAXI', 'oil', 'TE3', 
    'IXIC', 'HSI', 'FTSE', 'Dollar Index', 'DJI'
    ]

redundant_ixic = [
    'DAAA', 'DTB6', 'DTB4WK', 'DGS10', 'TE3', 'DE4', 
    'TE2', 'DE5', 'DTB3', 'DE6', 'EMA_20', 'CTB6M', 
    'CTB3M', 'EMA_50', 'CTB1Y', 'EMA_200', 'DGS5', 
    'S&P-F', 'FCHI', 'GSPC', 'GDAXI', 'oil', 'NYSE', 
    'HSI', 'FTSE', 'Dollar Index', 'TE6', 'DJI'
    ]

In [4]:
# Clean the data and save as feather
df_nyse_raw = pd.read_feather("/home/ubuntu/michael/MSc-Machine-Learning-Project/Datasets/Raw/combined_dataframe_NYSE.feather")
df_nyse_cleaned = clean_and_select_features_equities(df_nyse_raw, redundant_nyse)
df_nyse_cleaned.to_feather("/home/ubuntu/michael/MSc-Machine-Learning-Project/Datasets/Processed/cleaned_NYSE.feather")

df_ixic_raw = pd.read_feather("/home/ubuntu/michael/MSc-Machine-Learning-Project/Datasets/Raw/combined_dataframe_IXIC.feather")
df_ixic_cleaned = clean_and_select_features_equities(df_ixic_raw, redundant_ixic)
df_ixic_cleaned.to_feather("/home/ubuntu/michael/MSc-Machine-Learning-Project/Datasets/Processed/cleaned_IXIC.feather")


Shifting 21 columns by +1 day
Number of columns dropped: 34

Shifting 21 columns by +1 day
Number of columns dropped: 33


In [5]:
# Function to pre-process equities data
def preprocess_data_equities(feather_filepath: str, train_end: str = '2019-12-31', val_end: str = '2021-12-31'):
    """
    Loads data with date in YYYY-MM-DD format from an arrow file and performs a full pre-processing pipeline:
    1. Handles missing values using forward fill (no backward fill to avoid look-ahead bias);
    2. Drop any leading rows that remain NaNs after forward fill;
    3. Splits data chronologically into training, validation, and test sets;
    4. Normalise the data using StandardScaler fitted on the training set.
    Args:
        feather_filepath (str): The path to the Arrow file to be processed.
        train_end (str): The end date for the training set
        val_end (str): The end date for the validation set
    Returns:
        tuple: (train_df, val_df, test_df, scaler)
    """
    print(f"Beginning pre-processing of {feather_filepath}:")

    # Step 1: Load and sort the data
    df = feather.read_feather(feather_filepath)
    date_col = "Date"
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(date_col).reset_index(drop=True)
    print(f"Loaded data from {feather_filepath} with {df.shape[0]} rows and {df.shape[1]} columns.")

    # Step 2: Engineer target column: log-returns
    price_col = "Price" if "Price" in df.columns else "close"
    df["_log_price"] = np.log(df[price_col])
    df["ret_t"] = df["_log_price"].diff()
    df["y_next"]     = df["_log_price"].shift(-1) - df["_log_price"]
    df = df.dropna(subset=["y_next"]).reset_index(drop=True)

    # Step 3: Causal imputation (forward-fill only) and drop leading incomplete rows
    # Forward-fill all columns except data, target and log-price columns
    protect_cols = {date_col, "y_next", "_log_price"}
    ffill_cols = [c for c in df.columns if c not in protect_cols]
    df.loc[:, ffill_cols] = df[ffill_cols].ffill()

    # Build numeric feature set excluding target and log_price
    numeric_feature_cols = (
        df.select_dtypes(include=np.number).columns.difference(["y_next", "_log_price"])
    )
    # Drop columns that are entirely NaN (prevents false "no complete rows")
    all_nan_cols = [c for c in numeric_feature_cols if df[c].isna().all()]
    if all_nan_cols:
        print(f"Dropping {len(all_nan_cols)} all-NaN columns (e.g., {all_nan_cols[:5]})")
        df = df.drop(columns=all_nan_cols)
        numeric_feature_cols = [c for c in numeric_feature_cols if c not in all_nan_cols]

    if len(numeric_feature_cols) == 0:
        raise ValueError("No numeric feature columns remain after exclusions.")

    complete_mask = df[numeric_feature_cols].notna().all(axis=1)
    if not complete_mask.any():
        na_counts = df[numeric_feature_cols].isna().sum().sort_values(ascending=False).head(10)
        raise ValueError(f"After forward-fill, no rows have complete numeric features. Top NA columns:\n{na_counts}")
    first_complete_idx = complete_mask.idxmax()
    if first_complete_idx > 0:
        print(f"Dropping {first_complete_idx} leading rows with unresolved NaNs.")
    df = df.loc[first_complete_idx:].reset_index(drop=True)
    
    print("Missing values handled.")

    # Step 4: Chronological split using the provided end dates
    # Random splitting ignores the temporal order of the data, which is crucial for time series.
    # Here we split the data chronologically into training, validation, and test sets.
    train_end_date = pd.Timestamp(train_end)
    val_end_date = pd.Timestamp(val_end)
    train_df = df[df[date_col] <= train_end_date].copy()
    val_df = df[(df[date_col] > train_end_date) & (df[date_col] <= val_end_date)].copy()
    test_df = df[df[date_col] > val_end_date].copy()
    print("Split data chronologically:")
    print(f"Training set: {train_df.shape[0]} rows (<= {train_end_date.date()})")
    print(f"Validation set: {val_df.shape[0]} rows (until {val_end_date.date()})")
    print(f"Test set: {test_df.shape[0]} rows (after {val_end_date.date()})")

    # Step 5: Normalisation
    # Exclude target and log_price from scaling
    numeric_cols = df.select_dtypes(include=np.number).columns.difference(["y_next", "_log_price"])
    scaler = StandardScaler()
    scaler.fit(train_df[numeric_cols])
    print("Scaler fitted on training data.")
    # Transform the training, validation, and test sets
    train_df[numeric_cols] = scaler.transform(train_df[numeric_cols])
    val_df[numeric_cols] = scaler.transform(val_df[numeric_cols])
    test_df[numeric_cols] = scaler.transform(test_df[numeric_cols])

    return train_df, val_df, test_df, scaler

In [7]:
# Data pre-processing
if __name__ == "__main__":
    # Data directory
    data_dir = "/home/ubuntu/michael/MSc-Machine-Learning-Project/Datasets/"

    # Per-file cut-off dates
    split_params = {
        "Processed/cleaned_IXIC.feather":   dict(train_end="2019-08-12", val_end="2021-07-14"),
        "Processed/cleaned_NYSE.feather":   dict(train_end="2019-08-12", val_end="2021-07-14")
    }
    # Process each file
    for feather_file in ["Processed/cleaned_IXIC.feather",
                         "Processed/cleaned_NYSE.feather"]:
        feather_path = os.path.join(data_dir, feather_file)
        try:
            # Look up per-file dates (fallback to {} → defaults)
            params = split_params.get(feather_file, {})
            train_df, val_df, test_df, scaler = preprocess_data_equities(
                feather_path,
                **params  # expands to train_end=..., val_end=...
            )

            # Save processed splits
            stem = feather_file.replace(".feather", "")
            train_df.to_feather(os.path.join(data_dir, f"{stem}_train.feather"))
            val_df.to_feather(os.path.join(data_dir, f"{stem}_val.feather"))
            test_df.to_feather(os.path.join(data_dir, f"{stem}_test.feather"))
            print(f"Processed data saved for {feather_file}.\n")

        except FileNotFoundError as e:
            print(f"File not found: {e}\n")
        except Exception as e:
            print(f"An error occurred while processing {feather_file}: {e}\n")
        

Beginning pre-processing of /home/ubuntu/michael/MSc-Machine-Learning-Project/Datasets/Processed/cleaned_IXIC.feather:
Loaded data from /home/ubuntu/michael/MSc-Machine-Learning-Project/Datasets/Processed/cleaned_IXIC.feather with 3470 rows and 51 columns.
Dropping 1 leading rows with unresolved NaNs.
Missing values handled.
Split data chronologically:
Training set: 2417 rows (<= 2019-08-12)
Validation set: 484 rows (until 2021-07-14)
Test set: 567 rows (after 2021-07-14)
Scaler fitted on training data.
Processed data saved for Processed/cleaned_IXIC.feather.

Beginning pre-processing of /home/ubuntu/michael/MSc-Machine-Learning-Project/Datasets/Processed/cleaned_NYSE.feather:
Loaded data from /home/ubuntu/michael/MSc-Machine-Learning-Project/Datasets/Processed/cleaned_NYSE.feather with 3470 rows and 50 columns.
Dropping 1 leading rows with unresolved NaNs.
Missing values handled.
Split data chronologically:
Training set: 2417 rows (<= 2019-08-12)
Validation set: 484 rows (until 2021-0

In [14]:
df_check = pd.read_feather("/home/ubuntu/michael/MSc-Machine-Learning-Project/Datasets/Raw/BTC_USDT-5m.feather")
print(df_check.head())
date_col = "date"
if date_col in df_check.columns:
    df_check[date_col] = pd.to_datetime(df_check[date_col])
    print("Date range")
    print(f"   Start: {df_check[date_col].min()}")
    print(f"   End: {df_check[date_col].max()}")

                       date      open      high       low     close  \
0 2021-01-01 00:00:00+00:00  28923.63  29017.50  28913.12  28975.65   
1 2021-01-01 00:05:00+00:00  28975.65  28979.53  28846.28  28858.94   
2 2021-01-01 00:10:00+00:00  28858.94  28883.20  28690.17  28752.80   
3 2021-01-01 00:15:00+00:00  28752.80  28852.48  28720.91  28820.72   
4 2021-01-01 00:20:00+00:00  28822.17  28846.46  28744.09  28846.46   

       volume  
0  182.889878  
1  214.568104  
2  442.619587  
3  174.839779  
4  161.316784  
Date range
   Start: 2021-01-01 00:00:00+00:00
   End: 2021-12-31 23:55:00+00:00
